In [8]:
%run ../utils/init_env.py

In [9]:
import sqlite3
import json
import os
import pandas as pd
import nltk
nltk.download('punkt_tab')

import evaluation

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/atissera/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [10]:
DB_PATH = "/Users/atissera/Developer/repos/tesis/spider_data/test_database"
PREDICTIONS = "/Users/atissera/Developer/repos/tesis/spider_data/test.json"

In [14]:

kmaps = evaluation.build_foreign_key_map_from_json("../spider_data/test_tables.json")
evaluation.evaluate(gold="../data/gold_queries.sql", predict="../data/predicted_queries.sql", db_dir="../spider_data/test_database", etype="all", kmaps=kmaps)

eval_err_num:1
extra pred: SELECT Hangar.location, COUNT(PilotSkills.pilot_name) AS number_of_pilots, AVG(PilotSkills.age) AS average_age FROM Hangar JOIN PilotSkills ON Hangar.plane_name = PilotSkills.plane_name GROUP BY Hangar.location
extra gold: SELECT count(T1.pilot_name) ,  avg(T1.age) ,  T2.location FROM pilotskills AS T1 JOIN hangar AS T2 ON T1.plane_name  =  T2.plane_name GROUP BY T2.location

eval_err_num:2
hard pred: SELECT i.Institution_Name, COUNT(s.staff_ID) AS staff_count FROM institution i JOIN staff s ON i.Institution_ID = s.Institution_ID WHERE i.Founded > 1800 GROUP BY i.Institution_ID
hard gold: SELECT T1.institution_name ,  count(*) FROM institution AS T1 JOIN staff AS T2 ON T1.institution_id  =  T2.institution_id WHERE T1.founded  >  1800 GROUP BY T2.institution_id

eval_err_num:3
extra pred: SELECT DISTINCT d.Driver_Name FROM driver d JOIN race r1 ON d.Driver_ID = r1.Driver_ID JOIN race r2 ON d.Driver_ID = r2.Driver_ID WHERE r1.Pole_Position = 'James Hinchcliffe'

In [ ]:

with open(PREDICTIONS) as f:
    predictions = json.load(f)

for item in predictions[:5]:
    db_id = item["db_id"]
    query = item["query"]
    db_file = os.path.join(DB_PATH, db_id, db_id + ".sqlite")
    if not os.path.exists(db_file):
        print(f"❌ Database file not found: {db_file}")
        continue

    try:
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()
        cursor.execute(query)
        columns = [description[0] for description in cursor.description]  # get column names
        rows = cursor.fetchall()
        conn.close()

        result_df = pd.DataFrame(rows, columns=columns)
        print(f"📌 Database: {db_id} | Query:\n{query}")
        display(result_df)
    except Exception as e:
        print(f"❌ Error running query on {db_id}: {query}")
        print(e)

📌 Database: soccer_3 | Query:
SELECT count(*) FROM club


,count(*)
0,9


📌 Database: soccer_3 | Query:
SELECT count(*) FROM club


,count(*)
0,9


📌 Database: soccer_3 | Query:
SELECT Name FROM club ORDER BY Name ASC


,Name
0,Arsenal
1,Aston Villa
2,Blackburn Rovers
3,Bolton Wanderers
4,Chelsea
5,Everton
6,Fulham
7,Hull City
8,Liverpool


📌 Database: soccer_3 | Query:
SELECT Name FROM club ORDER BY Name ASC


,Name
0,Arsenal
1,Aston Villa
2,Blackburn Rovers
3,Bolton Wanderers
4,Chelsea
5,Everton
6,Fulham
7,Hull City
8,Liverpool


📌 Database: soccer_3 | Query:
SELECT Manager ,  Captain FROM club


,Manager,Captain
0,Arsène Wenger,Cesc Fàbregas
1,Martin O'Neill,Martin Laursen
2,Sam Allardyce,Ryan Nelsen
3,Gary Megson,Kevin Davies
4,Guus Hiddink,John Terry
5,David Moyes,Phil Neville
6,Roy Hodgson,Danny Murphy
7,Phil Brown,Ian Ashbee
8,Rafael Benítez,Steven Gerrard
